In [ ]:
import os
import time
import traceback

import pandas as pd
import httpx
from tqdm import tqdm

In [34]:
def fill_collection_info(collection_list):
    OPENSEA_API_KEY = ""

    __headers = {
        "accept": "application/json",
        "x-api-key": OPENSEA_API_KEY
    }

    __chain_map = {
        "klaytn": "klaytn",
        "ethereum": "ethereum",
        "polygon": "matic",
        "bnb": "bsc"
    }

    
    contract_lambda_url = lambda network, collection_id: f"https://api.opensea.io/api/v2/chain/{network}/contract/{collection_id}"
    collection_lambda_url = lambda version, slug: f"https://api.opensea.io/api/{version}/collection/{slug}"

    for collection in tqdm(collection_list):
        try:
            network = collection["network"]
            collection_id = collection["collection_id"]
            
            contract_url = contract_lambda_url(__chain_map[network], collection_id)
            contract_resp = httpx.get(contract_url, headers=__headers, timeout=5)
            contract_resp = contract_resp.json()
            
            slug = contract_resp["collection"]
            collection_resp_v1 = httpx.get(collection_lambda_url("v1", slug), headers=__headers, timeout=5)
            if collection_resp_v1.status_code != 200:
                collection_resp_v1 = {}
            else:
                collection_resp_v1 = collection_resp_v1.json()
                collection_resp_v1 = collection_resp_v1["collection"]
            
            collection_resp_v2 = httpx.get(collection_lambda_url("v2", slug), headers=__headers, timeout=5)
            if collection_resp_v2.status_code != 200:
                collection_resp_v2 = {}
            else:        
                collection_resp_v2 = collection_resp_v2.json()
            
            time.sleep(0.5)
            
            # collection["collection_name"] = contract_resp["name"]
            collection["nft_type"] = contract_resp.get("contract_standard")
            collection["creator"] = collection_resp_v1.get("owner")
            # collection["meta_uri"] = None
            collection["total_supply"] = collection_resp_v1.get("primary_asset_contracts", [{}])[0].get("total_supply", None)
            collection["deployment_date"] = collection_resp_v1.get("created_date")
            collection["slug"] = slug
            collection["category"] = collection_resp_v2.get("category")
            collection["banner_url"] = collection_resp_v1.get("banner_image_url")
            collection["isverified"] = collection_resp_v1.get("safelist_request_status") == "verified" if collection_resp_v1.get("safelist_request_status") is not None else None
            collection["discord_link"] = collection_resp_v2.get("discord_url")
            collection["twitter_link"] = f"https://twitter.com/{collection_resp_v2['twitter_username']}" if collection_resp_v2.get("twitter_username") not in ["", None] else None
            collection["instagram_link"] = f"https://www.instagram.com/{collection_resp_v2['instagram_username']}" if collection_resp_v2.get("instagram_username") not in ["", None] else None
            # collection["facebook_link"] = None
            collection["telegram_link"] = collection_resp_v1.get("telegram_url")
            collection["external_url"] = collection_resp_v1.get("external_url")
            # collection["producer"] = None
            collection["logo_url"] = collection_resp_v2.get("image_url")
            collection["description"] = collection_resp_v2.get("description")
            
        except:
            print(traceback.format_exc())
            continue
        
        
    return collection_list


In [ ]:
path = "./data/nft_collection_list_table_202311021300.csv"
collection_list = pd.read_csv(path).to_dict("records")

for batch in tqdm(range(400, len(collection_list), 100)):
    df = pd.DataFrame(fill_collection_info(collection_list[batch:batch+100]))
    df.to_csv(f"./output/collection_table_{batch}.csv", index=False)

In [6]:
path_output = "./output"
path_output_files = [os.path.join(path_output, f) for f in os.listdir(path_output)]
df = pd.concat([pd.read_csv(f) for f in path_output_files])

In [7]:
df

,banner_url,idx,network,collection_id,collection_name,slug,s3_logo_url,logo_url,update_date,nft_type,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gcs/files/f521398fce2bb2119...,24815,bnb,0xf601aa72650c6298e98a3027db32c734cba00720,Puff New Year,puff-new-year,NaN,NaN,2023-10-31 00:17:32,erc721,...,False,http://puffverse.pro,NaN,NaN,NaN,https://t.me/PuffverseChat,NaN,NaN,NaN,NaN
1,https://i.seadn.io/gcs/files/b525b53ac700361e2...,24816,bnb,0x3209b0cc1a9d15958bcfcb44714873024c2fdf8d,Cyberpunk Pi BSC,cyberpunk-pi-bnb,NaN,NaN,2023-10-31 00:17:32,erc721,...,False,https://space-pi.com,NaN,NaN,NaN,https://t.me/SpacePi_Com,NaN,NaN,NaN,NaN
2,https://i.seadn.io/gcs/files/34a15578cdfdc477d...,24817,bnb,0xc33d69a337b796a9f0f7588169cd874c3987bde9,Kryptomon - V1,kryptomonv1,NaN,NaN,2023-10-31 00:17:32,erc721,...,True,https://kryptomon.co,NaN,NaN,NaN,https://t.me/kryptomonofficial,NaN,NaN,NaN,NaN
3,https://i.seadn.io/gcs/files/717bb902372dbd42b...,24818,bnb,0xd5f2f405c087b2c55701e11b4e57bd17aeea29a5,Space Cartels Lands,space-cartels-lands-bnb,NaN,NaN,2023-10-31 00:17:32,erc721,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,24819,bnb,0xbb6717f336d0f36a5a86e42544a5d115da53c481,Gameta Dash Hippo Club,gameta-dash-hippo-club,NaN,NaN,2023-10-31 00:17:32,erc721,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,21799,klaytn,0xdbb43f4c44a66e6869911f2bfa49bf09fb4ac726,Klay Cosmodinos,klay-cosmodinos,klaytn/0e7d74f1f9f7f92e6356d4dea226ef4a1cfad7f...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,21800,klaytn,0x911c492a562be735e74f350ab8ffe52c0756afa2,KLAY CRESTs,klay-crests,klaytn/4dac3e8640f60d0e81ea3d4c215ea9fe73fb462...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,21801,klaytn,0x383af442e34168de088b67a27595f843f8a77c6d,Klay_Soonsoon,klay-soonsoon,klaytn/c61e75004bdcf5e15a995b8c73c42d1cca3d24b...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,21802,klaytn,0x162bac511ed282373824d59401f2e3d3648eae37,Klay-SoulCat,klay-soulcat,klaytn/0e054edd60329d3204690e18431a3e5d2b9fd94...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
del df["s3_logo_url"]

In [9]:
df

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gcs/files/f521398fce2bb2119...,24815,bnb,0xf601aa72650c6298e98a3027db32c734cba00720,Puff New Year,puff-new-year,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,http://puffverse.pro,NaN,NaN,NaN,https://t.me/PuffverseChat,NaN,NaN,NaN,NaN
1,https://i.seadn.io/gcs/files/b525b53ac700361e2...,24816,bnb,0x3209b0cc1a9d15958bcfcb44714873024c2fdf8d,Cyberpunk Pi BSC,cyberpunk-pi-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,https://space-pi.com,NaN,NaN,NaN,https://t.me/SpacePi_Com,NaN,NaN,NaN,NaN
2,https://i.seadn.io/gcs/files/34a15578cdfdc477d...,24817,bnb,0xc33d69a337b796a9f0f7588169cd874c3987bde9,Kryptomon - V1,kryptomonv1,NaN,2023-10-31 00:17:32,erc721,NaN,...,True,https://kryptomon.co,NaN,NaN,NaN,https://t.me/kryptomonofficial,NaN,NaN,NaN,NaN
3,https://i.seadn.io/gcs/files/717bb902372dbd42b...,24818,bnb,0xd5f2f405c087b2c55701e11b4e57bd17aeea29a5,Space Cartels Lands,space-cartels-lands-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,24819,bnb,0xbb6717f336d0f36a5a86e42544a5d115da53c481,Gameta Dash Hippo Club,gameta-dash-hippo-club,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,21799,klaytn,0xdbb43f4c44a66e6869911f2bfa49bf09fb4ac726,Klay Cosmodinos,klay-cosmodinos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,21800,klaytn,0x911c492a562be735e74f350ab8ffe52c0756afa2,KLAY CRESTs,klay-crests,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,21801,klaytn,0x383af442e34168de088b67a27595f843f8a77c6d,Klay_Soonsoon,klay-soonsoon,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,21802,klaytn,0x162bac511ed282373824d59401f2e3d3648eae37,Klay-SoulCat,klay-soulcat,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df["network"].value_counts()

network
polygon     4081
ethereum    3077
klaytn       619
bnb          266
Ethereum      10
Name: count, dtype: int64

In [13]:
df_not_updated = df.loc[df["isverified"].isna()]

In [15]:
df_not_updated["network"].value_counts()

network
polygon     2705
ethereum     733
klaytn       619
Ethereum      10
bnb            1
Name: count, dtype: int64

In [17]:
df_not_updated_Ethereum = df_not_updated.loc[df_not_updated["network"] == "Ethereum"]
df_not_updated_Ethereum

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
51,NaN,23055,Ethereum,0xFA7E3F898c80E31A3aedeAe8b0C713a3F9666264,Akuma Origins,akumaorigins,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,23058,Ethereum,0xF8C18Df7509c03b45e6247b2b9E73fcaDEF24dd6,Bitmates,bitmates,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,24628,Ethereum,0x6d0de90CDc47047982238fcF69944555D27Ecb25,Regulars,regulars,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.651016e+09,NaN,NaN,NaN
76,NaN,24480,Ethereum,0xf3E6DbBE461C6fa492CeA7Cb1f5C5eA660EB1B47,FameLadySquad,fameladysquad,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.626138e+09,NaN,NaN,NaN
84,NaN,24488,Ethereum,0xE106C63E655dF0E300b78336af587F300Cff9e76,OxyaOriginProject,oxyaoriginproject,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.641532e+09,NaN,NaN,NaN
85,NaN,24489,Ethereum,0x3598Fff0f78Dd8b497e12a3aD91FeBcFC8F49d9E,Private Jet Pyjama Party,pjpp,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.675524e+09,NaN,NaN,NaN
44,NaN,23348,Ethereum,0xe39Fb076bd6Be0bbf400db0dfaD28104fDc5FF46,ALPACADABRAZ 3D,alpacadabraz-3d,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,23349,Ethereum,0x47f3A38990ca12E39255E959F7D97fBE5906Afd4,Ape Reunion,apereunion,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,24026,Ethereum,0x5be99338289909d6dbBC57bb791140Ef85cCbCAB,Hausphases by Fabrik,hausphasesbyfabrik,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.698657e+09,NaN,NaN,NaN
70,NaN,24174,Ethereum,0xd1d411D2Da363144248b98aDaB453aa3B19cCF04,Rug Radio Membership Pass,rug-radio-membership-pass,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.640609e+09,NaN,NaN,NaN


In [19]:
df_not_updated_Ethereum["network"] = df_not_updated_Ethereum["network"].apply(lambda x: "ethereum")
df_not_updated_Ethereum

/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_64076/529800139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_updated_Ethereum["network"] = df_not_updated_Ethereum["network"].apply(lambda x: "ethereum")


,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
51,NaN,23055,ethereum,0xFA7E3F898c80E31A3aedeAe8b0C713a3F9666264,Akuma Origins,akumaorigins,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,23058,ethereum,0xF8C18Df7509c03b45e6247b2b9E73fcaDEF24dd6,Bitmates,bitmates,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,24628,ethereum,0x6d0de90CDc47047982238fcF69944555D27Ecb25,Regulars,regulars,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.651016e+09,NaN,NaN,NaN
76,NaN,24480,ethereum,0xf3E6DbBE461C6fa492CeA7Cb1f5C5eA660EB1B47,FameLadySquad,fameladysquad,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.626138e+09,NaN,NaN,NaN
84,NaN,24488,ethereum,0xE106C63E655dF0E300b78336af587F300Cff9e76,OxyaOriginProject,oxyaoriginproject,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.641532e+09,NaN,NaN,NaN
85,NaN,24489,ethereum,0x3598Fff0f78Dd8b497e12a3aD91FeBcFC8F49d9E,Private Jet Pyjama Party,pjpp,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.675524e+09,NaN,NaN,NaN
44,NaN,23348,ethereum,0xe39Fb076bd6Be0bbf400db0dfaD28104fDc5FF46,ALPACADABRAZ 3D,alpacadabraz-3d,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,23349,ethereum,0x47f3A38990ca12E39255E959F7D97fBE5906Afd4,Ape Reunion,apereunion,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,24026,ethereum,0x5be99338289909d6dbBC57bb791140Ef85cCbCAB,Hausphases by Fabrik,hausphasesbyfabrik,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.698657e+09,NaN,NaN,NaN
70,NaN,24174,ethereum,0xd1d411D2Da363144248b98aDaB453aa3B19cCF04,Rug Radio Membership Pass,rug-radio-membership-pass,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.640609e+09,NaN,NaN,NaN


In [20]:
df_not_updated_Ethereum

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
51,NaN,23055,ethereum,0xFA7E3F898c80E31A3aedeAe8b0C713a3F9666264,Akuma Origins,akumaorigins,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,NaN,23058,ethereum,0xF8C18Df7509c03b45e6247b2b9E73fcaDEF24dd6,Bitmates,bitmates,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,24628,ethereum,0x6d0de90CDc47047982238fcF69944555D27Ecb25,Regulars,regulars,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.651016e+09,NaN,NaN,NaN
76,NaN,24480,ethereum,0xf3E6DbBE461C6fa492CeA7Cb1f5C5eA660EB1B47,FameLadySquad,fameladysquad,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.626138e+09,NaN,NaN,NaN
84,NaN,24488,ethereum,0xE106C63E655dF0E300b78336af587F300Cff9e76,OxyaOriginProject,oxyaoriginproject,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.641532e+09,NaN,NaN,NaN
85,NaN,24489,ethereum,0x3598Fff0f78Dd8b497e12a3aD91FeBcFC8F49d9E,Private Jet Pyjama Party,pjpp,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.675524e+09,NaN,NaN,NaN
44,NaN,23348,ethereum,0xe39Fb076bd6Be0bbf400db0dfaD28104fDc5FF46,ALPACADABRAZ 3D,alpacadabraz-3d,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,23349,ethereum,0x47f3A38990ca12E39255E959F7D97fBE5906Afd4,Ape Reunion,apereunion,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,24026,ethereum,0x5be99338289909d6dbBC57bb791140Ef85cCbCAB,Hausphases by Fabrik,hausphasesbyfabrik,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.698657e+09,NaN,NaN,NaN
70,NaN,24174,ethereum,0xd1d411D2Da363144248b98aDaB453aa3B19cCF04,Rug Radio Membership Pass,rug-radio-membership-pass,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.640609e+09,NaN,NaN,NaN


In [21]:
temp = fill_collection_info(df_not_updated_Ethereum.to_dict("records"))
temp = pd.DataFrame(temp)
temp

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gae/PcXVcun_WdVPzsol0l2mBko...,23055,ethereum,0xFA7E3F898c80E31A3aedeAe8b0C713a3F9666264,Akuma Origins,akumaorigins,None,NaN,erc721,None,...,True,https://www.akumaorigins.com,None,None,NaN,None,NaN,NaN,None,None
1,https://i.seadn.io/gae/DjF6bD2Z-70XHxKyMJOdbj_...,23058,ethereum,0xF8C18Df7509c03b45e6247b2b9E73fcaDEF24dd6,Bitmates,bitmates,None,NaN,erc721,None,...,True,https://bitmates.io,None,None,NaN,None,NaN,NaN,None,None
2,https://i.seadn.io/gae/aJnj_vWfizRBysBWzml0Ias...,24628,ethereum,0x6d0de90CDc47047982238fcF69944555D27Ecb25,Regulars,regulars,None,NaN,erc721,None,...,True,https://regular.world,None,None,NaN,None,1.651016e+09,NaN,None,None
3,https://i.seadn.io/gae/Sd6knb_KwwnRqmUJ6dcT_lc...,24480,ethereum,0xf3E6DbBE461C6fa492CeA7Cb1f5C5eA660EB1B47,FameLadySquad,fameladysquad,None,NaN,erc721,None,...,True,https://www.fameladysquad.com/,None,None,NaN,None,1.626138e+09,NaN,None,None
4,https://i.seadn.io/gcs/files/0fa19c69504fa11df...,24488,ethereum,0xE106C63E655dF0E300b78336af587F300Cff9e76,OxyaOriginProject,oxyaoriginproject,None,NaN,erc721,None,...,True,http://www.oxyaorigin.com,None,None,NaN,None,1.641532e+09,NaN,None,None
5,https://i.seadn.io/gae/DeSf4hHZyH6j9-1aSuf14sW...,24489,ethereum,0x3598Fff0f78Dd8b497e12a3aD91FeBcFC8F49d9E,Private Jet Pyjama Party,pjpp,None,NaN,erc721,None,...,True,http://www.pjpp.io,None,None,NaN,None,1.675524e+09,NaN,None,None
6,https://i.seadn.io/gae/jLIZxvwZkavEWcuWEkuO8J0...,23348,ethereum,0xe39Fb076bd6Be0bbf400db0dfaD28104fDc5FF46,ALPACADABRAZ 3D,alpacadabraz-3d,None,NaN,erc721,None,...,True,https://alpacadabraz.io/,None,None,NaN,https://t.me/alpacadabraz,NaN,NaN,None,None
7,https://i.seadn.io/gcs/files/1c68870f69232404a...,23349,ethereum,0x47f3A38990ca12E39255E959F7D97fBE5906Afd4,Ape Reunion,apereunion,None,NaN,erc721,None,...,True,https://www.apereunion.xyz/#/,None,None,NaN,None,NaN,NaN,None,None
8,https://i.seadn.io/gcs/files/a8704e6e394e73591...,24026,ethereum,0x5be99338289909d6dbBC57bb791140Ef85cCbCAB,Hausphases by Fabrik,hausphasesbyfabrik,None,NaN,erc721,None,...,False,https://gate.fabrik.world/,None,None,NaN,None,1.698657e+09,NaN,None,None
9,https://i.seadn.io/gae/e7k5z8bajMs_siuo_Y3pw0x...,24174,ethereum,0xd1d411D2Da363144248b98aDaB453aa3B19cCF04,Rug Radio Membership Pass,rug-radio-membership-pass,None,NaN,erc721,None,...,True,https://www.rug.fm/,None,None,NaN,None,1.640609e+09,NaN,None,None


In [22]:
df_updated = df.loc[~df["isverified"].isna()]
df_updated

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gcs/files/f521398fce2bb2119...,24815,bnb,0xf601aa72650c6298e98a3027db32c734cba00720,Puff New Year,puff-new-year,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,http://puffverse.pro,NaN,NaN,NaN,https://t.me/PuffverseChat,NaN,NaN,NaN,NaN
1,https://i.seadn.io/gcs/files/b525b53ac700361e2...,24816,bnb,0x3209b0cc1a9d15958bcfcb44714873024c2fdf8d,Cyberpunk Pi BSC,cyberpunk-pi-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,https://space-pi.com,NaN,NaN,NaN,https://t.me/SpacePi_Com,NaN,NaN,NaN,NaN
2,https://i.seadn.io/gcs/files/34a15578cdfdc477d...,24817,bnb,0xc33d69a337b796a9f0f7588169cd874c3987bde9,Kryptomon - V1,kryptomonv1,NaN,2023-10-31 00:17:32,erc721,NaN,...,True,https://kryptomon.co,NaN,NaN,NaN,https://t.me/kryptomonofficial,NaN,NaN,NaN,NaN
3,https://i.seadn.io/gcs/files/717bb902372dbd42b...,24818,bnb,0xd5f2f405c087b2c55701e11b4e57bd17aeea29a5,Space Cartels Lands,space-cartels-lands-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,24819,bnb,0xbb6717f336d0f36a5a86e42544a5d115da53c481,Gameta Dash Hippo Club,gameta-dash-hippo-club,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,https://i.seadn.io/gae/OwbT8ElOLBj2Y9GK-t4jhCV...,21748,ethereum,0xd83b6f9a3e623ae4427298726ae32907b477b8cc,[the reliquary],thereliquary,NaN,NaN,erc721,NaN,...,False,https://vibes.art/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,https://i.seadn.io/gcs/files/7caceb0f4b6b09e92...,21749,ethereum,0xf699b8f6244b81195756f20dbbdcbed1f66078bc,Editions-II by TunART,tunarteditions2,NaN,NaN,erc1155,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,21750,ethereum,0xf38b19a7418699e7c69b66aeaa8618b467950bd7,SariyildizOE,unidentified-contract-b9rxw5ldke,NaN,NaN,erc1155,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,https://i.seadn.io/gcs/files/0c4572d34d9f2ace2...,21751,ethereum,0x14e645e2bb0e15c87230c6206e2c85b494c4a116,Wastelands.cult,wastelandscult,NaN,NaN,erc721,NaN,...,False,https://wastelands.studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_updated = pd.concat([df_updated, temp])
df_updated

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gcs/files/f521398fce2bb2119...,24815,bnb,0xf601aa72650c6298e98a3027db32c734cba00720,Puff New Year,puff-new-year,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,http://puffverse.pro,NaN,NaN,NaN,https://t.me/PuffverseChat,NaN,NaN,NaN,NaN
1,https://i.seadn.io/gcs/files/b525b53ac700361e2...,24816,bnb,0x3209b0cc1a9d15958bcfcb44714873024c2fdf8d,Cyberpunk Pi BSC,cyberpunk-pi-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,https://space-pi.com,NaN,NaN,NaN,https://t.me/SpacePi_Com,NaN,NaN,NaN,NaN
2,https://i.seadn.io/gcs/files/34a15578cdfdc477d...,24817,bnb,0xc33d69a337b796a9f0f7588169cd874c3987bde9,Kryptomon - V1,kryptomonv1,NaN,2023-10-31 00:17:32,erc721,NaN,...,True,https://kryptomon.co,NaN,NaN,NaN,https://t.me/kryptomonofficial,NaN,NaN,NaN,NaN
3,https://i.seadn.io/gcs/files/717bb902372dbd42b...,24818,bnb,0xd5f2f405c087b2c55701e11b4e57bd17aeea29a5,Space Cartels Lands,space-cartels-lands-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,24819,bnb,0xbb6717f336d0f36a5a86e42544a5d115da53c481,Gameta Dash Hippo Club,gameta-dash-hippo-club,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,https://i.seadn.io/gae/DeSf4hHZyH6j9-1aSuf14sW...,24489,ethereum,0x3598Fff0f78Dd8b497e12a3aD91FeBcFC8F49d9E,Private Jet Pyjama Party,pjpp,None,NaN,erc721,None,...,True,http://www.pjpp.io,None,None,NaN,None,1.675524e+09,NaN,None,None
6,https://i.seadn.io/gae/jLIZxvwZkavEWcuWEkuO8J0...,23348,ethereum,0xe39Fb076bd6Be0bbf400db0dfaD28104fDc5FF46,ALPACADABRAZ 3D,alpacadabraz-3d,None,NaN,erc721,None,...,True,https://alpacadabraz.io/,None,None,NaN,https://t.me/alpacadabraz,NaN,NaN,None,None
7,https://i.seadn.io/gcs/files/1c68870f69232404a...,23349,ethereum,0x47f3A38990ca12E39255E959F7D97fBE5906Afd4,Ape Reunion,apereunion,None,NaN,erc721,None,...,True,https://www.apereunion.xyz/#/,None,None,NaN,None,NaN,NaN,None,None
8,https://i.seadn.io/gcs/files/a8704e6e394e73591...,24026,ethereum,0x5be99338289909d6dbBC57bb791140Ef85cCbCAB,Hausphases by Fabrik,hausphasesbyfabrik,None,NaN,erc721,None,...,False,https://gate.fabrik.world/,None,None,NaN,None,1.698657e+09,NaN,None,None


In [24]:
df_not_updated_bnb = df_not_updated.loc[df_not_updated["network"] == "bnb"]
df_not_updated_bnb

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
21,NaN,25329,bnb,0x9e347b9a9c68ed24b2fc3bee4cdffbe5985dc7bf,Alchemy Pay Easter Egg NFT,alchemy-pay-easter-egg-nft,NaN,2023-10-31 00:17:32,erc721,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
temp = fill_collection_info(df_not_updated_bnb.to_dict("records"))
temp = pd.DataFrame(temp)
temp

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,None,25329,bnb,0x9e347b9a9c68ed24b2fc3bee4cdffbe5985dc7bf,Alchemy Pay Easter Egg NFT,alchemy-pay-easter-egg-nft,None,2023-10-31 00:17:32,erc721,None,...,None,None,None,None,NaN,None,NaN,NaN,None,None


In [26]:
df_updated = pd.concat([df_updated, temp])
df_updated

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gcs/files/f521398fce2bb2119...,24815,bnb,0xf601aa72650c6298e98a3027db32c734cba00720,Puff New Year,puff-new-year,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,http://puffverse.pro,NaN,NaN,NaN,https://t.me/PuffverseChat,NaN,NaN,NaN,NaN
1,https://i.seadn.io/gcs/files/b525b53ac700361e2...,24816,bnb,0x3209b0cc1a9d15958bcfcb44714873024c2fdf8d,Cyberpunk Pi BSC,cyberpunk-pi-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,https://space-pi.com,NaN,NaN,NaN,https://t.me/SpacePi_Com,NaN,NaN,NaN,NaN
2,https://i.seadn.io/gcs/files/34a15578cdfdc477d...,24817,bnb,0xc33d69a337b796a9f0f7588169cd874c3987bde9,Kryptomon - V1,kryptomonv1,NaN,2023-10-31 00:17:32,erc721,NaN,...,True,https://kryptomon.co,NaN,NaN,NaN,https://t.me/kryptomonofficial,NaN,NaN,NaN,NaN
3,https://i.seadn.io/gcs/files/717bb902372dbd42b...,24818,bnb,0xd5f2f405c087b2c55701e11b4e57bd17aeea29a5,Space Cartels Lands,space-cartels-lands-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,24819,bnb,0xbb6717f336d0f36a5a86e42544a5d115da53c481,Gameta Dash Hippo Club,gameta-dash-hippo-club,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,https://i.seadn.io/gae/jLIZxvwZkavEWcuWEkuO8J0...,23348,ethereum,0xe39Fb076bd6Be0bbf400db0dfaD28104fDc5FF46,ALPACADABRAZ 3D,alpacadabraz-3d,NaN,NaN,erc721,NaN,...,True,https://alpacadabraz.io/,NaN,NaN,NaN,https://t.me/alpacadabraz,NaN,NaN,NaN,NaN
7,https://i.seadn.io/gcs/files/1c68870f69232404a...,23349,ethereum,0x47f3A38990ca12E39255E959F7D97fBE5906Afd4,Ape Reunion,apereunion,NaN,NaN,erc721,NaN,...,True,https://www.apereunion.xyz/#/,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
8,https://i.seadn.io/gcs/files/a8704e6e394e73591...,24026,ethereum,0x5be99338289909d6dbBC57bb791140Ef85cCbCAB,Hausphases by Fabrik,hausphasesbyfabrik,NaN,NaN,erc721,NaN,...,False,https://gate.fabrik.world/,NaN,NaN,NaN,None,1.698657e+09,NaN,NaN,NaN
9,https://i.seadn.io/gae/e7k5z8bajMs_siuo_Y3pw0x...,24174,ethereum,0xd1d411D2Da363144248b98aDaB453aa3B19cCF04,Rug Radio Membership Pass,rug-radio-membership-pass,NaN,NaN,erc721,NaN,...,True,https://www.rug.fm/,NaN,NaN,NaN,None,1.640609e+09,NaN,NaN,NaN


In [28]:
df_not_updated_cypress = df_not_updated.loc[df_not_updated["network"] == "klaytn"]
df_not_updated_cypress

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
40,NaN,24864,klaytn,0x70df6bd1b521d62fd92c2c546468b37f03496183,Monster Shoe Club,monster-shoe-club,NaN,2023-10-31 00:17:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.698661e+09,NaN,NaN,NaN
41,NaN,24866,klaytn,0xc8db437ff4ee5bee6ecb58652c28b20edd7d89e2,Puuvilla Crystal ball,puuvilla-crystalball,NaN,2023-10-31 00:17:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NaN,24867,klaytn,0x0cf20fa0ef8e3228c052aed348cec77b385d5eae,Towner Membership,towner-membership,NaN,2023-10-31 00:17:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.678878e+09,NaN,NaN,NaN
43,NaN,24869,klaytn,0xa9a95c5fef43830d5d67156a2582a2e793acb465,Klip Drops,klipdrops,NaN,2023-10-31 00:17:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,NaN,24871,klaytn,0x4cd320595123c89cb05a7b8a26b9f09490ee65c5,create your SM6,create-your-sm6,NaN,2023-10-31 00:17:32,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,21799,klaytn,0xdbb43f4c44a66e6869911f2bfa49bf09fb4ac726,Klay Cosmodinos,klay-cosmodinos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,21800,klaytn,0x911c492a562be735e74f350ab8ffe52c0756afa2,KLAY CRESTs,klay-crests,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,21801,klaytn,0x383af442e34168de088b67a27595f843f8a77c6d,Klay_Soonsoon,klay-soonsoon,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,21802,klaytn,0x162bac511ed282373824d59401f2e3d3648eae37,Klay-SoulCat,klay-soulcat,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
temp = fill_collection_info(df_not_updated_cypress.to_dict("records"))
temp = pd.DataFrame(temp)
temp

Traceback (most recent call last):
  File "/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_64076/94231763.py", line 29, in fill_collection_info
    slug = contract_resp["collection"]
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^
KeyError: 'collection'

Traceback (most recent call last):
  File "/Users/user/.pyenv/versions/3.11.5/envs/lambda/lib/python3.11/site-packages/httpcore/_exceptions.py", line 10, in map_exceptions
    yield
  File "/Users/user/.pyenv/versions/3.11.5/envs/lambda/lib/python3.11/site-packages/httpcore/_backends/sync.py", line 126, in read
    return self._sock.recv(max_bytes)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/user/.pyenv/versions/3.11.5/lib/python3.11/ssl.py", line 1296, in recv
    return self.read(buflen)
           ^^^^^^^^^^^^^^^^^
  File "/Users/user/.pyenv/versions/3.11.5/lib/python3.11/ssl.py", line 1169, in read
    return self._sslobj.read(len)
           ^^^^^^^^^^^^^^^^^^^^^^
TimeoutError: The read operation timed out

The above ex

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gcs/files/926f3059d1d28c194...,24864,klaytn,0x70df6bd1b521d62fd92c2c546468b37f03496183,Monster Shoe Club,monster-shoe-club,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,None,NaN,NaN,NaN,None,1.698661e+09,NaN,NaN,NaN
1,https://i.seadn.io/gcs/files/1468fe538b25ed97e...,24866,klaytn,0xc8db437ff4ee5bee6ecb58652c28b20edd7d89e2,Puuvilla Crystal ball,puuvilla-crystalball,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,https://puuvillasociety.com/,NaN,NaN,NaN,https://t.me/puuvillasociety,NaN,NaN,NaN,NaN
2,https://i.seadn.io/gcs/files/998f60f233f57adc9...,24867,klaytn,0x0cf20fa0ef8e3228c052aed348cec77b385d5eae,Towner Membership,towner-membership,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,http://nfttown.io,NaN,NaN,NaN,None,1.678878e+09,NaN,NaN,NaN
3,https://i.seadn.io/gcs/files/b3df3d161aff1f9a3...,24869,klaytn,0xa9a95c5fef43830d5d67156a2582a2e793acb465,Klip Drops,klipdrops,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
4,None,24871,klaytn,0x4cd320595123c89cb05a7b8a26b9f09490ee65c5,create your SM6,create-your-sm6,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,https://i.seadn.io/gae/w2qMOk0xkgqQpOfpKJLupOg...,21799,klaytn,0xdbb43f4c44a66e6869911f2bfa49bf09fb4ac726,Klay Cosmodinos,klay-cosmodinos,NaN,NaN,erc721,NaN,...,False,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
615,None,21800,klaytn,0x911c492a562be735e74f350ab8ffe52c0756afa2,KLAY CRESTs,klay-crests,NaN,NaN,erc1155,NaN,...,False,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
616,https://i.seadn.io/gae/vJQxvs2OuvLpeEAx3NUZomh...,21801,klaytn,0x383af442e34168de088b67a27595f843f8a77c6d,Klay_Soonsoon,klay-soonsoon,NaN,NaN,erc1155,NaN,...,False,http://soonsoons.com,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
617,https://i.seadn.io/gae/mxaCP4SVq5bpVHTU9z3oWTL...,21802,klaytn,0x162bac511ed282373824d59401f2e3d3648eae37,Klay-SoulCat,klay-soulcat,NaN,NaN,erc721,NaN,...,False,https://linktr.ee/Klay_SoulCat,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [32]:
df_updated = pd.concat([df_updated, temp])
df_updated

/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_64076/3500870991.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_updated = pd.concat([df_updated, temp])


,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
0,https://i.seadn.io/gcs/files/f521398fce2bb2119...,24815,bnb,0xf601aa72650c6298e98a3027db32c734cba00720,Puff New Year,puff-new-year,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,http://puffverse.pro,NaN,NaN,NaN,https://t.me/PuffverseChat,NaN,NaN,NaN,NaN
1,https://i.seadn.io/gcs/files/b525b53ac700361e2...,24816,bnb,0x3209b0cc1a9d15958bcfcb44714873024c2fdf8d,Cyberpunk Pi BSC,cyberpunk-pi-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,https://space-pi.com,NaN,NaN,NaN,https://t.me/SpacePi_Com,NaN,NaN,NaN,NaN
2,https://i.seadn.io/gcs/files/34a15578cdfdc477d...,24817,bnb,0xc33d69a337b796a9f0f7588169cd874c3987bde9,Kryptomon - V1,kryptomonv1,NaN,2023-10-31 00:17:32,erc721,NaN,...,True,https://kryptomon.co,NaN,NaN,NaN,https://t.me/kryptomonofficial,NaN,NaN,NaN,NaN
3,https://i.seadn.io/gcs/files/717bb902372dbd42b...,24818,bnb,0xd5f2f405c087b2c55701e11b4e57bd17aeea29a5,Space Cartels Lands,space-cartels-lands-bnb,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,24819,bnb,0xbb6717f336d0f36a5a86e42544a5d115da53c481,Gameta Dash Hippo Club,gameta-dash-hippo-club,NaN,2023-10-31 00:17:32,erc721,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,https://i.seadn.io/gae/w2qMOk0xkgqQpOfpKJLupOg...,21799,klaytn,0xdbb43f4c44a66e6869911f2bfa49bf09fb4ac726,Klay Cosmodinos,klay-cosmodinos,NaN,NaN,erc721,NaN,...,False,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
615,None,21800,klaytn,0x911c492a562be735e74f350ab8ffe52c0756afa2,KLAY CRESTs,klay-crests,NaN,NaN,erc1155,NaN,...,False,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
616,https://i.seadn.io/gae/vJQxvs2OuvLpeEAx3NUZomh...,21801,klaytn,0x383af442e34168de088b67a27595f843f8a77c6d,Klay_Soonsoon,klay-soonsoon,NaN,NaN,erc1155,NaN,...,False,http://soonsoons.com,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
617,https://i.seadn.io/gae/mxaCP4SVq5bpVHTU9z3oWTL...,21802,klaytn,0x162bac511ed282373824d59401f2e3d3648eae37,Klay-SoulCat,klay-soulcat,NaN,NaN,erc721,NaN,...,False,https://linktr.ee/Klay_SoulCat,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [ ]:
df_updated.to_csv("./output/collection_table_20231103.csv", index=False)

In [35]:
df_not_updated_eth = df_not_updated.loc[df_not_updated["network"] == "ethereum"]
df_not_updated_eth

,banner_url,idx,network,collection_id,collection_name,slug,logo_url,update_date,nft_type,creator,...,isverified,external_url,discord_link,instagram_link,facebook_link,telegram_link,transaction_timestamp,cetegory,twitter_link,description
11,NaN,21115,ethereum,john-wingfields-editions,John Wingfield's Editions,john-wingfields-editions,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,21116,ethereum,0x29f0adc2f81bff599b18ba636ccab4cb9512cf36,LiveLikeADog,livelikeadog,NaN,NaN,erc721,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,21128,ethereum,0xebf39b3ee65a7fb265918af8f94010083d14fd37,Rough Terrain Containers,rough-terrain-by-containers-v2,NaN,NaN,erc721,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,23215,ethereum,0x97cf7b49a3d51ed077c5346e6e226c8eb0c750fa,3x724k7,3x724k7,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,23216,ethereum,0x16376312cca9e7e2f15f6eae12a86b72df2b5f61,Boots By Jeremy Booth,boots-vn1ytsxkja,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NaN,23905,ethereum,0xf9ae12ddba6cbeb7930489119f4612cc42d5a3b2,YAKANHIKOU - OFFICIAL,yakanhikou-official,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,23502,ethereum,0xbadad1ebfa38ea6d226e591233fb1f148d0c413b,Project: 2023 Cats Gang,2023catsgang-nft,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,23503,ethereum,0x8ba65ba38baad2d108f8a64877d9dbedc427627a,A Year in Portraits by Jeremy Cowart,a-year-in-portraits,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,24164,ethereum,0xf2545537d19d7bcba206a43f928aa18a61c242f5,KPMachines,kp-machines-v2,NaN,NaN,erc721,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.678128e+09,NaN,NaN,NaN


In [36]:
temp = fill_collection_info(df_not_updated_eth.to_dict("records"))
temp = pd.DataFrame(temp)
temp

  0%|          | 1/733 [00:00<04:10,  2.93it/s]

Traceback (most recent call last):
  File "/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_64076/1927272682.py", line 29, in fill_collection_info
    slug = contract_resp["collection"]
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^
KeyError: 'collection'



  1%|          | 6/733 [00:06<13:14,  1.09s/it]

Traceback (most recent call last):
  File "/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_64076/1927272682.py", line 29, in fill_collection_info
    slug = contract_resp["collection"]
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^
KeyError: 'collection'



  1%|          | 9/733 [00:10<12:36,  1.05s/it]

...

 12%|█▏        | 86/733 [02:05<14:26,  1.34s/it]

Traceback (most recent call last):
  File "/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_64076/1927272682.py", line 29, in fill_collection_info
    slug = contract_resp["collection"]
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^
KeyError: 'collection'



 14%|█▍        | 106/733 [02:39<15:45,  1.51s/it]

Traceback (most recent call last):
  File "/var/folders/k3/2w4zylcs0l12r34ky2dtqg5h0000gn/T/ipykernel_64076/1927272682.py", line 29, in fill_collection_info
    slug = contract_resp["collection"]
           ~~~~~~~~~~~~~^^^^^^^^^^^^^^
KeyError: 'collection'



 21%|██        | 155/733 [04:01<16:10,  1.68s/it]